# 🌐 Python `requests` Module — Examples + Short Exercises

* This notebook demonstrates how to use the **`requests`** library to interact with HTTP endpoints and APIs

We’ll cover
- Basic GET and POST requests  
- Query parameters and headers  
- JSON responses  
- Error handling  
- Timeouts  
- Sessions (persistent headers/cookies)

> **Install** (if needed): `pip install requests`

## 📦 Imports

We import `requests` for HTTP calls and `json` for optional formatting.

In [ ]:
import requests
import json

## 1️⃣ Basic GET Request
* sends an HTTP **GET** request to a test endpoint (`https://httpbin.org/get`),   
prints the status code and parses the JSON response
* a GET request is used to retrieve data from a server, without changing anything on the server (it’s the most common and safest type of HTTP request)

In [ ]:
url = "https://httpbin.org/get"
response = requests.get(url)

print("Status code:", response.status_code)
print("Response JSON:")
print(json.dumps(response.json(), indent=4))

**Exercise 1**  
- change `url` to `https://httpbin.org/uuid` and print the JSON  


## 2️⃣ Query Parameters
* passes query parameters via __`params`__
* __`requests`__ encodes and appends them to the URL

In [ ]:
params = {"name": "Alice", "city": "Boston"}
response = requests.get("https://httpbin.org/get", params=params)

print("Requested URL:", response.url)
print("JSON response:")
print(json.dumps(response.json(), indent=4))

**Exercise 2**  
- Add a parameter with a **space** in it (e.g., `"favorite movie": "The Matrix"`).  
- Verify that `response.url` shows the parameter URL-encoded
- Access the echoed params in `response.json()["args"]`

## 3️⃣ Custom Headers
* sends a custom `User-Agent` header
* useful for auth tokens, content types, and polite identification

In [ ]:
headers = {"User-Agent": "Jupyter-Demo/1.0"}
response = requests.get("https://httpbin.org/headers", headers=headers)

print("Response JSON:")
print(json.dumps(response.json(), indent=4))

**Exercise 3**  
- Add a fake API token header, e.g., `{"Authorization": "Bearer MYTOKEN"}`
- Confirm it appears under `response.json()["headers"]`

## 4️⃣ POST Request (Form Data and JSON)
* sends data with **POST** in two ways: form data (like a web form, i.e., key=value pairs separated by &) and JSON

In [ ]:
# Form data
data = {"username": "dave", "password": "secret"}
response = requests.post("https://httpbin.org/post", data=data)
print("Form POST:")
print(response.json()["form"])

# JSON data
json_data = {"course": "Python", "level": "AI Engineering", 
                "meta": {
                    "instructor": "Dave",
                    "duration_days": 5
                }
            }
response = requests.post("https://httpbin.org/post", json=json_data)
print("\nJSON POST:")
print(json.dumps(response.json()["json"], indent=4))

**Exercise 4**  
- Change the form keys/values and verify they echo back under `"form"`.  
- Add a nested object to `json_data` (e.g., `{"meta": {"instructor": "Dave"}}`) and confirm it appears under `"json"`.

## 5️⃣ Handling Errors

**What this does:**  
Requests a `404` endpoint and uses `.raise_for_status()` to throw `HTTPError` for 4xx/5xx codes.

In [ ]:
try:
    bad_response = requests.get("https://httpbin.org/status/404")
    bad_response.raise_for_status()
except requests.exceptions.HTTPError as e:
    print("Caught an HTTPError:", e)

**Exercise 5**  
- Try other status endpoints like `https://httpbin.org/status/500` and `418`.  
- Wrap them with `try/except` and print a custom message depending on the status code.

## 6️⃣ JSON Response Handling (Real API Example)
* calls the GitHub API for the `psf/requests` repository and prints a few fields  
* public APIs can rate-limit; if you see errors, try later

In [ ]:
response = requests.get("https://api.github.com/repos/psf/requests")
data = response.json()

print("Repository name:", data.get("name"))
print("Stars:", data.get("stargazers_count"))
print("Open issues:", data.get("open_issues_count"))

**Exercise 6**  
- Print the repo `license` name if present (nested under `data["license"]["name"]`).  
- Fetch and print the top 3 open issues titles from `https://api.github.com/repos/psf/requests/issues` (hint: it's a list of JSON objects).

In [ ]:
# License (may be None if not provided)
license_info = data.get("license")

if license_info:
    print("License:", license_info.get("name"))
else:
    print("License: None")

# --- Part 2: Get top 3 open issues ---
issues_url = "https://api.github.com/repos/psf/requests/issues"
issues_response = requests.get(issues_url)
issues = issues_response.json()

print("\nTop 3 open issue titles:")
for issue in issues[:3]:
    print("-", issue["title"])

## 7️⃣ Timeouts

**What this does:**  
Requests a slow endpoint and enforces `timeout=2` seconds to avoid hanging.

In [ ]:
try:
    response = requests.get("https://httpbin.org/delay/3", timeout=2)
    print("Status:", response.status_code)
except requests.exceptions.Timeout:
    print("Request timed out!")

**Exercise 7**  
- Increase the timeout to `5` and confirm the call succeeds

## 8️⃣ Using a Session

**What this does:**  
Uses a `Session` to persist headers and cookies across requests (e.g., staying “logged in”).

In [ ]:
with requests.Session() as session:
    session.headers.update({"User-Agent": "SessionExample/1.0"})
    r1 = session.get("https://httpbin.org/cookies/set/demo_cookie/value123")
    r2 = session.get("https://httpbin.org/cookies")
    print("Session cookies:", r2.json())

**Exercise 8**  
- Add another cookie via `https://httpbin.org/cookies/set/another_cookie/xyz` and verify it appears

In [ ]:
# %% [markdown]
# ## 9️⃣ HTTP Methods Overview
# 
# These are the main HTTP verbs you’ll use when working with web APIs:
# 
# | Method | Typical Use | Description |
# |---------|--------------|-------------|
# | **GET** | Retrieve data | Fetches information without changing it |
# | **POST** | Create data | Sends new data to the server |
# | **PUT** | Replace data | Updates or replaces an existing resource |
# | **PATCH** | Modify data | Partially updates an existing resource |
# | **DELETE** | Remove data | Deletes a resource |
# 
# Let’s try each method using the https://httpbin.org endpoints.

# %%
import requests

# 1️⃣ GET — retrieve data
r_get = requests.get("https://httpbin.org/get", params={"demo": "true"})
print("GET:", r_get.status_code)
print(r_get.json()["args"])

# %%
# 2️⃣ POST — send data (create new resource)
r_post = requests.post("https://httpbin.org/post", json={"name": "Alice", "role": "Instructor"})
print("POST:", r_post.status_code)
print(r_post.json()["json"])

# %%
# 3️⃣ PUT — replace existing resource
r_put = requests.put("https://httpbin.org/put", json={"name": "Alice", "role": "Admin"})
print("PUT:", r_put.status_code)
print(r_put.json()["json"])

# %%
# 4️⃣ PATCH — partially update existing resource
r_patch = requests.patch("https://httpbin.org/patch", json={"role": "Lead Instructor"})
print("PATCH:", r_patch.status_code)
print(r_patch.json()["json"])

# %%
# 5️⃣ DELETE — delete a resource
r_delete = requests.delete("https://httpbin.org/delete")
print("DELETE:", r_delete.status_code)
print(r_delete.json()["url"])